#ANN with MLFLOW 

# ============================================================
#                  INSTALLING ANN with MLFLOW
# ============================================================

# Logging experiments and registering models with MLflow

# Loading a logged model for inference using the MLflow PyFunc flavor

# Viewing experiment results in the MLflow UI

# Comparing runs, selecting the best model, and deploying it as a REST API

# Running a hyperparameter sweep on the training script

# Comparing hyperparameter sweep results in the MLflow UI

# Selecting the best-performing model and deploying it as a REST API

# Building a container image suitable for deployment to a cloud platform
# ============================================================

In [14]:

import pandas as pd
import numpy as np
import keras 
from hyperopt import STATUS_OK, Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature 

In [15]:
## load raw dataset
data=pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
    sep=";", 
)   

In [16]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [17]:
# split data into training and validation and test sets
train,test = train_test_split(data, test_size = 0.25, random_state=42)

In [18]:
train["quality"].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [19]:

train_x=train.drop(["quality"], axis=1).values

train_y=train[["quality"]].values.ravel()

# Validation data set
test_x=test.drop(["quality"], axis=1).values
test_y=test["quality"].values.ravel() 

# splitting this train into train and validation

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.20, random_state=42)
signature = infer_signature(train_x, train_y)
                                                
                                                   



In [20]:
## ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y,test_x,test_y):

    ## Define ANN model architecture 
    mean=np.mean(train_x, axis=0)
    var=np.var(train_x, axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## the compile ANN model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train ANN model with lr and momentum params with MLFLOW tracking
    
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y, validation_data=(valid_x,valid_y),
                    epochs=epochs,
                    batch_size= 64)
        
        ## Evaluate the model

        eval_result=model.evaluate(valid_x, valid_y,batch_size=64)
        eval_rmse=eval_result[1]

        ## Log the parameters and result
        mlflow.log_params(params)    
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## the log the model and schem of out put and input
        mlflow.tensorflow.log_model(model,"model", signature=signature)  

        return {"loss":eval_rmse, "status": STATUS_OK, "model":model}  



In [21]:
## Create objective function with hyperopt

def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [22]:
## Loguinform with lrand momentum ranges 
space={
    "lr":hp.loguniform("lr",np.log(1e-6),np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)

}

In [23]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 367ms/step - loss: 34.3399 - root_mean_squared_error: 5.8600
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.7040 - root_mean_squared_error: 5.8910   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 34.4134 - root_mean_squared_error: 5.8663 - val_loss: 34.0354 - val_root_mean_squared_error: 5.8340

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 32.1579 - root_mean_squared_error: 5.6708
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 33.7979 - root_mean_squared_error: 5.8134 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.3621 - root_mean_squared_error: 5.7760 - val_loss: 32.9903 - val_root_mean_squared_error: 5.7437

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 32.8489 - root_mean_squared_error: 5.7314
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.9598 - root_mean_squared_error: 5.7

2026/01/18 12:44:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 376ms/step - loss: 34.9552 - root_mean_squared_error: 5.9123
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.5168 - root_mean_squared_error: 5.9595   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 35.1256 - root_mean_squared_error: 5.9267 - val_loss: 34.4643 - val_root_mean_squared_error: 5.8706

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 35.6378 - root_mean_squared_error: 5.9697
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.1425 - root_mean_squared_error: 5.8430 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.3765 - root_mean_squared_error: 5.7772 - val_loss: 32.7036 - val_root_mean_squared_error: 5.7187

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 33.4230 - root_mean_squared_error: 5.7813
44/46 ━━━━

2026/01/18 12:44:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 377ms/step - loss: 32.8303 - root_mean_squared_error: 5.7298
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18.6223 - root_mean_squared_error: 4.2459   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 9.1675 - root_mean_squared_error: 3.0278 - val_loss: 2.4188 - val_root_mean_squared_error: 1.5553

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.6759 - root_mean_squared_error: 1.6358
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.1792 - root_mean_squared_error: 1.4757 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9763 - root_mean_squared_error: 1.4058 - val_loss: 1.8263 - val_root_mean_squared_error: 1.3514

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.5790 - root_mean_squared_error: 1.6059
37/46 ━━━━━━━━━━━

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 374ms/step - loss: 34.9045 - root_mean_squared_error: 5.9080
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.3974 - root_mean_squared_error: 5.5992   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 25.5147 - root_mean_squared_error: 5.0512 - val_loss: 16.4654 - val_root_mean_squared_error: 4.0578

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 16.2986 - root_mean_squared_error: 4.0372
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.1104 - root_mean_squared_error: 3.7525 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.3209 - root_mean_squared_error: 3.3647 - val_loss: 7.2012 - val_root_mean_squared_error: 2.6835

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 7.9181 - root_mean_squared_error: 2.8139
42/46 ━━━

2026/01/18 12:45:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:47<00:00, 11.85s/trial, best loss: 1.2404913902282715]


2026/01/18 12:45:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters: {'lr': np.float64(0.002932816477652215), 'momentum': np.float64(0.43311477969276935)}
Best eval rmse: 1.2404913902282715


In [24]:
## Inferencing the best ANN model

from mlflow.models import validate_serving_input

model_uri = 'runs:/c7e263dee8e24d3fb3a01b930f4672c4/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.7450557],
       [6.9318457],
       [6.501051 ],
       ...,
       [6.3322077],
       [6.938834 ],
       [5.6138268]], shape=(1225, 1), dtype=float32)

In [25]:
# Load model as a PyFuncModel.
model_uri = 'runs:/c7e263dee8e24d3fb3a01b930f4672c4/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.7450557],
       [6.9318457],
       [6.501051 ],
       ...,
       [6.3322077],
       [6.938834 ],
       [5.6138268]], shape=(1225, 1), dtype=float32)

In [26]:
## Register in the ANN model registry
mlflow.register_model(model_uri,"wine-quality")

Registered model 'wine-quality' already exists. Creating a new version of this model...
2026/01/18 12:45:36 WARNING mlflow.tracking._model_registry.fluent: Run with id c7e263dee8e24d3fb3a01b930f4672c4 has no artifacts at artifact path 'model', registering model based on models:/m-41a60ded84704e259a02f3e020d6867c instead
Created version '3' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1768711536980, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1768711536980, metrics=None, model_id=None, name='wine-quality', params=None, run_id='c7e263dee8e24d3fb3a01b930f4672c4', run_link=None, source='models:/m-41a60ded84704e259a02f3e020d6867c', status='READY', status_message=None, tags={}, user_id=None, version=3>